In [ ]:
# numpy for numerics, sympy for algebra, plotly and matplotlib for plots
import numpy as np
import cupy as cp
from sympy import *
from sympy.abc import x, t
from IPython.display import display, Math
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import numpy as np

plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 10})

# Gaussian Derivative using FFT

## 1D

Calculate the derivative of a 1D FFT numerically and analytically and plot to compare.

In [ ]:
class Gaussian1D_Analytic:
  def __init__(self):
    self.x, self.x0, self.sigma = symbols('x x0 sigma')
    self.n = exp(-(self.x-self.x0)**2/(2*self.sigma**2))
    self.dndx = diff(self.n, self.x).simplify()
    self.d2ndx2 = diff(self.n, self.x, 2).simplify()

gaussian1D_Analytic = Gaussian1D_Analytic()
Math(r'n = {}, '.format(latex(gaussian1D_Analytic.n))+r'dn/dx = {}, '.format(latex(gaussian1D_Analytic.dndx))+r'd^2n/dx^2 = {}'.format(latex(gaussian1D_Analytic.d2ndx2)))

For $n = e^{-(x-x_0)^2/(2 \sigma^2)}$, $dn/dx = -(x - x_0)/\sigma^2 \cdot n$.

### Parameters

In [ ]:
class Gaussian1D_Parms:
  def __init__(self):
    self.x0 = 1
    self.xmin = -10
    self.xmax = 10
    self.sigma = 2

### Analytic Derivative

In [ ]:
class Gaussian1D_AnalyticNumeric:
  def __init__(self):
    self.analytic = Gaussian1D_Analytic()
    self.parms = Gaussian1D_Parms()
    self.pts = 1000
    self.x = np.linspace(self.parms.xmin, self.parms.xmax, self.pts)
    self.dxdt_num = lambdify(self.analytic.x, self.analytic.dndx.subs([(self.analytic.x0, self.parms.x0), (self.analytic.sigma, self.parms.sigma)]))

  def Get_dxdt(self):
    return self.dxdt_num(self.x)

gaussian1D_AnalyticNumeric = Gaussian1D_AnalyticNumeric()
plt.plot(gaussian1D_AnalyticNumeric.x, gaussian1D_AnalyticNumeric.Get_dxdt())
plt.xlabel('x')
plt.ylabel('$dn/dx, n=e^{-(x-x_0)^2/(2 \sigma^2)}$')
plt.title('Analytic Derivative')
plt.show()

### Numeric Derivative via FFT

In [ ]:
class Gaussian1D_Numeric:
  def __init__(self):
    self.parms = Gaussian1D_Parms()
    self.LX  = 100
    self.dx = (self.parms.xmax-self.parms.xmin)/self.LX
    self.x = np.linspace(self.parms.xmin, self.parms.xmax, self.LX)
    self.phi = np.exp(-(self.x-self.parms.x0)**2/(2*self.parms.sigma**2))

  def GetNumericDerivative(self):
    kappa = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(self.phi.shape[0], d=self.dx))
    phi_hat = np.fft.fftshift(np.fft.fft(self.phi))
    dphi_hat = I*kappa*phi_hat
    dphi = np.fft.ifft(np.fft.ifftshift(dphi_hat))
    return dphi

### Compare Analytic and Numeric by Plotting

In [ ]:
gaussian1D_Numeric = Gaussian1D_Numeric()
dxdt_num = gaussian1D_Numeric.GetNumericDerivative()
plt.scatter(gaussian1D_Numeric.x, gaussian1D_Numeric.GetNumericDerivative(), label='Numeric')
plt.plot(gaussian1D_AnalyticNumeric.x, gaussian1D_AnalyticNumeric.Get_dxdt(), label='Analytic')
plt.xlabel('x')
plt.ylabel('$dn/dx, n=e^{-(x-x_0)^2/(2 \sigma^2)}$')
plt.title('Analytic Derivative vs. Numeric Derivative')
plt.legend()
plt.show()

## 2D

Calculate the derivative of a 1D FFT numerically and analytically and plot to compare.

In [ ]:
class Gaussian2D_Analytic:
  def __init__(self):
    self.x, self.y, self.x0, self.y0, self.sigma = symbols('x y x0 y0 sigma')
    self.n = exp(-((self.x-self.x0)**2 + (self.y-self.y0)**2)/(2*self.sigma**2))
    self.d2ndx2 = (diff(self.n, self.x, 2) + diff(self.n, self.y, 2)).simplify()
    self.d4ndx4 = (diff(self.d2ndx2, self.x, 2) + diff(self.d2ndx2, self.y, 2)).simplify()

gaussian2D_Analytic = Gaussian2D_Analytic()
Math(r'n = {}, '.format(latex(gaussian2D_Analytic.n))+r'd^2n/dx^2 = {}, '.format(latex(gaussian2D_Analytic.d2ndx2))+r'd^4n/dx^4 = {}'.format(latex(gaussian2D_Analytic.d4ndx4)))

### Parameters

In [ ]:
class Gaussian2D_Parms:
  def __init__(self):
    self.x0 = 1
    self.y0 = 0.5
    self.xmin = -10
    self.xmax = 10
    self.ymin = -10
    self.ymax = 10
    self.sigma = 2
    self.pts = 100

### Analytic Derivative

In [ ]:
class Gaussian2D_AnalyticNumeric:
  def __init__(self):
    self.analytic = Gaussian2D_Analytic()
    self.parms = Gaussian2D_Parms()
    self.Init()

  def Init(self):
    self.pts = self.parms.pts
    self.x = cp.linspace(self.parms.xmin, self.parms.xmax, self.pts)
    self.y = cp.linspace(self.parms.ymin, self.parms.ymax, self.pts)
    self.X, self.Y = cp.meshgrid(self.x, self.y)
    self.d2ndx2_num = lambdify((self.analytic.x, self.analytic.y), self.analytic.d2ndx2.subs([(self.analytic.x0, self.parms.x0), (self.analytic.y0, self.parms.y0), (self.analytic.sigma, self.parms.sigma)]))
    self.d4ndx4_num = lambdify((self.analytic.x, self.analytic.y), self.analytic.d4ndx4.subs([(self.analytic.x0, self.parms.x0), (self.analytic.y0, self.parms.y0), (self.analytic.sigma, self.parms.sigma)]))

  def Get_d2dx2(self):
    return self.d2ndx2_num(self.X, self.Y)#.reshape(self.pts, self.pts)

  def Get_d4dx4(self):
    return self.d4ndx4_num(self.X, self.Y)#.reshape(self.pts, self.pts)

gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_AnalyticNumeric.X.get()
Y = gaussian2D_AnalyticNumeric.Y.get()
Z = gaussian2D_AnalyticNumeric.Get_d2dx2().get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()


In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_AnalyticNumeric.X.get()
Y = gaussian2D_AnalyticNumeric.Y.get()
Z = gaussian2D_AnalyticNumeric.Get_d4dx4().get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^4 \phi \text{: 2D Guassian Fourth Derivative}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()


### Numeric Derivative via FFT

In [ ]:
class Gaussian2D_Numeric:
  def __init__(self):
    self.parms = Gaussian2D_Parms()
    self.Init()

  def Init(self):
    # Define grid
    self.nx = self.parms.pts
    self.ny = self.parms.pts
    self.dx = (self.parms.xmax-self.parms.xmin)/self.nx
    self.dy = (self.parms.ymax-self.parms.ymin)/self.ny
    x = cp.linspace(self.parms.xmin, self.parms.xmax, self.nx)
    y = cp.linspace(self.parms.ymin, self.parms.ymax, self.ny)
    self.x, self.y = cp.meshgrid(x, y)

    # Create the wave numbers
    kx = cp.fft.fftfreq(self.nx, d=self.dx) * 2 * cp.pi
    ky = cp.fft.fftfreq(self.ny, d=self.dy) * 2 * cp.pi
    self.kx, self.ky = cp.meshgrid(kx, ky)
    self.k2 = self.kx**2 + self.ky**2
    self.k4 = self.k2**2#self.kx**4 + self.ky**4 + 2*(self.kx**2 + self.ky**2)

    self.phi = cp.exp(-((self.x-self.parms.x0)**2 + (self.y-self.parms.y0)**2)/(2*self.parms.sigma**2))

    self.Analytic = Gaussian2D_AnalyticNumeric()
    self.Analytic.parms.pts = self.nx
    self.Analytic.Init()

  def Get_d2dx2(self):
    return self.Analytic.Get_d2dx2()

  def Get_d4dx4(self):
    return self.Analytic.Get_d4dx4()

  def Get_nabla2_Fourier(self, phi):
    phi_hat = cp.fft.fft2(phi)
    d2phi_hat = -self.k2*phi_hat
    d2phi = cp.fft.ifft2(d2phi_hat)
    return d2phi.real#.get()

  def Get_nabla4_Fourier(self, phi):
    phi_hat = cp.fft.fft2(phi)
    d4phi_hat = +self.k4*phi_hat
    d4phi = cp.fft.ifft2(d4phi_hat)
    return d4phi.real#.get()

  def Get_nabla2_FiniteDiff(self, phi):
    return ((cp.roll(phi, 1, axis=0) + cp.roll(phi, -1, axis=0) - 2*phi)/self.dx**2 + (cp.roll(phi, 1, axis=1) + cp.roll(phi, -1, axis=1) - 2*phi)/self.dy**2)#.get()

  def Get_nabla4_FiniteDiff(self, phi):
    return (
        (cp.roll(phi, 2, axis=0) - 4 * cp.roll(phi, 1, axis=0) + 6 * phi - 4 * cp.roll(phi, -1, axis=0) + cp.roll(phi, -2, axis=0))/self.dx**4 +
        (cp.roll(phi, 2, axis=1) - 4 * cp.roll(phi, 1, axis=1) + 6 * phi - 4 * cp.roll(phi, -1, axis=1) + cp.roll(phi, -2, axis=1))/self.dy**4
    )#.get()

gaussian2D_numeric = Gaussian2D_Numeric()

print(gaussian2D_numeric.dx)

In [ ]:
%timeit gaussian2D_numeric.Get_nabla2_Fourier(gaussian2D_numeric.phi)
%timeit gaussian2D_numeric.Get_nabla4_Fourier(gaussian2D_numeric.phi)

%timeit gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi)
%timeit gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi)

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla2_Fourier(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot_r')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative - Fourier}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla2_FiniteDiff(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot_r')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative - Finite Differences}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla4_Fourier(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Fourth Derivative - Fourier}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Fourth Derivative - Finite Differences}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

# Vacancy PFC

Energy function $f$:

$$
f(\rho) = \frac{\rho}{2} \left[ r + (1 + \nabla^2)^2 \right] \rho + \frac{\rho^4}{4}
$$

Then energy functional $F$:

$$
F(\rho) = \int d \vec{x} \left( \frac{\rho}{2} \left[ r + (1 + \nabla^2)^2 \right] \rho + \frac{\rho^4}{4} \right)
$$

And the Caan-Hilliard equation:

$$
\frac{\partial \rho}{\partial t} = \nabla^2 \frac{\delta F}{\delta \rho} = \nabla^2 \left[ (r+1)\rho + \rho^3 + 2 \nabla^2 \rho + \nabla^4 \rho \right]
$$

In Fourier space gives:

$$
\frac{\partial \hat{\rho}}{\partial t} = -\left[ k^6 - 2k^4 + (r+1)k^2 \right] \hat{\rho} - k^2 \hat{(\rho^3)}
$$

## Model

In [ ]:
class PFC2D_Vacancy_Parms:
  def __init__(self):
    self.r = None      # corresponds to temperature
    self.nu = None     # number of pixels in the square region
    self.ppu = None    # pixels per unit
    self.rho0 = None   # the average density of the initial field
    self.eta = None    # the strength of the gaussian noise added to each time step
    self.dt = None     # time step
    self.seed = 0      # set for reproducibility

class PFC2D_Vacancy:
  def __init__(self):
    self.t = None      # elapsed time
    self.parms = PFC2D_Vacancy_Parms()

  def InitParms(self):
    # Set seed for reproducibility
    cp.random.seed(cp.uint64(self.parms.seed))

    # Define grid
    self.nx = self.parms.nu
    self.ny = self.parms.nu
    lu = 2 * np.pi * self.parms.nu / self.parms.ppu
    self.dx = 2 * np.pi / self.parms.ppu   # pixel size in real space
    self.dy = 2 * np.pi / self.parms.ppu   # pixel size in real space
    x = cp.linspace(0, lu, self.nx)
    y = cp.linspace(0, lu, self.ny)
    self.x, self.y = cp.meshgrid(x, y)

    # using self.x, self.y create an array of ordered pairs
    self.r = cp.array([self.x.flatten(), self.y.flatten()]).T.reshape(self.nx, self.ny, 2)

    # Create the wave numbers
    kx = cp.fft.fftfreq(self.nx, d=self.dx) * 2 * cp.pi
    ky = cp.fft.fftfreq(self.ny, d=self.dy) * 2 * cp.pi
    self.kx, self.ky = cp.meshgrid(kx, ky)
    self.k2 = self.kx**2 + self.ky**2
    self.k4 = self.k2**2
    self.k6 = self.k2**3
    self.lincoeff = (-self.k2 * (self.parms.r + 1) + 2 * self.k4 - self.k6)
    self.t = 0

    # SetDT
    self.SetDT(self.parms.dt)

  def SetDT(self, dt):
    self.parms.dt = dt
    self.expcoeff = cp.exp(self.lincoeff*self.parms.dt)

    self.expcoeff_nonlin = cp.ones_like(self.x) * self.parms.dt
    self.expcoeff_nonlin[self.lincoeff != 0] = (self.expcoeff[self.lincoeff != 0] - 1)/self.lincoeff[self.lincoeff != 0]

    self.expcoeff_nonlin2 = cp.zeros_like(self.x)
    self.expcoeff_nonlin2[self.lincoeff != 0] = (self.expcoeff[self.lincoeff != 0] - (1 + self.lincoeff[self.lincoeff != 0]*self.parms.dt))/cp.power(self.lincoeff[self.lincoeff != 0],2)

  def GetEtaNoise(self):
    noise = cp.random.normal(0, 1, (self.nx, self.ny))
    noise_fft = cp.fft.fft2(noise)

    # zero out the zero-frequency component to remove the mean
    noise_fft[0, 0] = 0

    # inverse fourier to get normalized noise
    noise = cp.fft.ifft2(noise_fft).real

    # give noise an absolute amplitude of 1
    noise /= cp.max(noise) - cp.min(noise)
    return noise*self.parms.eta

  def InitFieldFlat(self, noisy = True):
    self.phi = self.parms.rho0 * cp.ones((self.nx, self.ny))
    self.phi += self.GetEtaNoise() if noisy & (self.parms.eta != 0) else 0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi0 = cp.fft.ifft2(self.phi_hat).real

  def InitFieldCrystal(self, noisy = True):
    q1 = cp.array([-np.sqrt(3)*0.5, -0.5])
    q2 = cp.array([0.0, 1.0])
    q3 = cp.array([np.sqrt(3)*0.5, -0.5])

    self.phi = self.parms.rho0 * (cp.cos(self.r.dot(q1)) + cp.cos(self.r.dot(q2)) + cp.cos(self.r.dot(q3))) /6 + self.parms.rho0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi_hat[0,0] = self.nx*self.ny*self.parms.rho0
    self.phi = cp.fft.ifft2(self.phi_hat).real
    self.phi += self.GetEtaNoise() if noisy & (self.parms.eta != 0) else 0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi0 = cp.fft.ifft2(self.phi_hat).real

  def AddNoise(self):
    self.phi += self.GetEtaNoise()
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi0 = cp.fft.ifft2(self.phi_hat).real

  def TimeStep(self):
    # vacancy terms
    self.vac = -6*1500*cp.power(self.phi,2)*(self.phi < 0)
    self.vac_hat = cp.fft.fft2(self.vac)

    # phi^3 term
    self.phi3 = cp.power(self.phi, 3)
    self.phi3_hat = cp.fft.fft2(self.phi3)

    self.phi_hat = self.phi_hat + self.parms.dt * (self.lincoeff * self.phi_hat - self.k2 * (self.phi3_hat + self.vac_hat))
    self.phi = cp.fft.ifft2(self.phi_hat).real
    self.t += self.parms.dt

  def TimeStepCross(self):
    self.phi_hat = cp.fft.fft2(self.phi)

    self.N0_hat = self.Get_N_hat(self.phi)

    self.phi_hat0 = self.expcoeff * self.phi_hat + -self.k2 * self.expcoeff_nonlin * self.N0_hat
    self.phi0 = cp.fft.ifft2(self.phi_hat0).real

    self.N1_hat = (self.Get_N_hat(self.phi0) - self.N0_hat)/self.parms.dt

    phi_hat1 = self.expcoeff * self.phi_hat + -self.k2 * (self.expcoeff_nonlin * self.N0_hat + self.expcoeff_nonlin2 * self.N1_hat)
    phi1 = cp.fft.ifft2(phi_hat1).real

    delta_phi = phi1 - self.phi0
    if delta_phi.max() - delta_phi.min() > 0.01:
      self.phi0 = phi1
      self.N1_hat = (self.Get_N_hat(self.phi0) - self.N0_hat)/self.parms.dt

      delta_phi = phi1 - self.phi0
      if delta_phi.max() - delta_phi.min() > 0.01:
        raise Exception("predictor-corrector failed")

    # throw an exception if phi max > 2
    if cp.max(phi1) > 2:
      raise Exception("phi max > 2")

    self.phi_hat = phi_hat1
    self.phi = phi1
    self.t += self.parms.dt

  def Get_N_hat(self, phi):
    # vacancy energy
    self.vac = -6*1500*cp.power(phi,2)*(phi < 0) # 3 ϕ (|ϕ| - ϕ) = { -6ϕ^2 where ϕ < 0, or, 0 where ϕ > 0 }
    # vac_hat = cp.fft.fft2(self.vac)

    # phi^3 term
    phi3 = cp.power(phi, 3)
    # phi3_hat = cp.fft.fft2(phi3)

    return cp.fft.fft2(phi3 + self.vac)

  def Get_d2ndxdy(self, phi):
    phi_hat = cp.fft.fft2(phi)
    d2phi_hat = -self.k2*phi_hat
    d2phi = cp.fft.ifft2(d2phi_hat)
    return d2phi.real

  def Get_d2ndxdy_FiniteDiff(self, phi):
    return (cp.roll(phi, 1, axis=0) + cp.roll(phi, -1, axis=0) + cp.roll(phi, 1, axis=1) + cp.roll(phi, -1, axis=1) - 4*phi)/self.dx**2

## Plotting

In [ ]:
def PlotTimeStep(_pfc):
  # vacancy terms
  _phi_hat = _pfc.phi_hat.get()
  _phi = _pfc.phi.get()
  _lincoeff = _pfc.lincoeff.get()
  _k2 = _pfc.k2.get()
  _dt = _pfc.parms.dt

  _vac = 6*1500*np.power(_phi,2)*(_phi < 0)
  _vac_hat = np.fft.fft2(_vac)

  # phi^3 term
  _phi3 = np.power(_phi, 3)
  _phi3_hat = np.fft.fft2(_phi3)

  _dphi_hat_dt = _dt * (_lincoeff * _phi_hat - _k2 * (_phi3_hat + _vac_hat))

  plt.figure(figsize=(8, 8))

  plt.subplot(4, 2, 1)
  plt.title("$\phi$")
  plt.imshow(_phi, cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 2)
  plt.title("$\hat{\phi}$")
  plt.contourf(np.abs(_phi_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 3)
  plt.title("vac")
  plt.contourf(np.abs(_vac), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 4)
  plt.title("$\hat{vac}$")
  plt.contourf(np.abs(_vac_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 5)
  plt.title("$\phi^3$")
  plt.contourf(np.abs(_phi3), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 6)
  plt.title("$\hat{\phi}^3$")
  plt.contourf(np.abs(_phi3_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  # self.phi_hat = self.phi_hat + self.parms.dt * (self.lincoeff * self.phi_hat - self.k2 * (self.phi3_hat + self.vac_hat))
  plt.subplot(4, 2, 7)
  plt.title("$t1$")
  plt.contourf(np.abs(_lincoeff * _phi_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 8)
  plt.title("$t2$")
  plt.contourf(np.abs(_k2 * (_phi3_hat + _vac_hat)), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.tight_layout()
  plt.show()

## Saving / Restoring

In [ ]:
def SaveData(_pfc, _filename):
  np.savez(_filename + f'_nu!{_pfc.parms.nu}_rho0!{_pfc.parms.rho0:0.2f}_seed!{_pfc.parms.seed}_t!{_pfc.t:0.2f}', parms=_pfc.parms.__dict__, t=_pfc.t, phi=_pfc.phi.get())

In [ ]:
def ReadPFCFile(_filename):
  data = np.load(_filename, allow_pickle=True)
  pfc = PFC2D_Vacancy()
  pfc.parms.__dict__ = data['parms'].item()
  pfc.InitParms()
  pfc.phi = cp.array(data['phi'])
  pfc.phi_hat = cp.fft.fft2(pfc.phi)
  pfc.phi0 = cp.fft.ifft2(pfc.phi_hat).real
  pfc.t = data['t']
  return pfc

## Testing

In [ ]:
pfc = PFC2D_Vacancy()
pfc.parms.r = -1.6
pfc.parms.nu = 400
pfc.parms.ppu = 10
pfc.parms.rho0 = 0.1
pfc.parms.eta = 0.2
pfc.parms.dt = 3e-3
pfc.parms.seed = 2

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(30000):
  try:
    # if i % 100 == 0:
    #   pfc.AddNoise()
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

SaveData(pfc, 'flat')

In [ ]:
_pfc = ReadPFCFile('flat_nu!100_rho0!0.10_seed!1_t!90.00.npz')
fig = go.Figure(data=go.Heatmap(z=_pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
# show the image scaled to 1 px per point
for i in range(30000):
  try:
    # if i % 100 == 0:
    #   _pfc.AddNoise()
    _pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(_pfc.t, _pfc.phi.mean(), _pfc.phi.min(), _pfc.phi.max(), np.power((_pfc.phi - _pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=_pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=200,
    height=200,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

### Debug

In [ ]:
pfc = PFC2D_Vacancy()
pfc.parms.r = -1.6
pfc.parms.nu = 100
pfc.parms.ppu = 10
pfc.parms.rho0 = 0.3
pfc.parms.eta = 0.7
pfc.parms.dt = 1e-2

pfc.InitParms()
pfc.InitFieldCrystal()
pfc.phi_hat[0,0]

In [ ]:
pfc.lincoeff[0,0], pfc.expcoeff[0,0], pfc.expcoeff_nonlin[0,0], pfc.expcoeff_nonlin2[0,0]

In [ ]:
pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max()

In [ ]:
pfc.phi = _phi
pfc.phi_hat = _phi_hat
pfc.t = _t
pfc.parms.dt = 3e-10 # worked at 2.6 broke at 2.7
for i in range(300):
  pfc.TimeStep()
  pfc.phi_hat[50,50]=0
PlotTimeStep(pfc)
print(pfc.phi.min(), pfc.phi.max(), pfc.phi.mean(), pfc.t)

In [ ]:
pfc.parms.dt = 3e-10 # worked at 2.6 broke at 2.7
for i in range(1):
  pfc.TimeStep()
PlotTimeStep(pfc)
print(pfc.phi.min(), pfc.phi.max(), pfc.phi.mean(), pfc.t)

In [ ]:
plt.imshow(np.log(np.abs(pfc.phi_hat)))

In [ ]:
pfc.phi_hat[50,50]

In [ ]:
pfc.kx[50,49:52], pfc.ky[50,49:52]

In [ ]:
pfc.TimeStep()
pfc.lincoeff[49:52,49:52], -pfc.k2[49:52,49:52], pfc.phi_hat[49:52,49:52], pfc.phi3_hat[49:52,49:52]

In [ ]:
pfc.lincoeff

In [ ]:
expcoeff = np.exp(pfc.lincoeff*pfc.parms.dt)
expcoeff.min(), expcoeff.max()

expcoeff_nonlin = np.ones_like(expcoeff) * pfc.parms.dt
expcoeff_nonlin[pfc.lincoeff != 0] = (expcoeff[pfc.lincoeff != 0] - 1)/pfc.lincoeff[pfc.lincoeff != 0]
expcoeff_nonlin.min(), expcoeff_nonlin.max()

In [ ]:
expcoeff_nonlin = np.ones_like(expcoeff) * pfc.parms.dt
expcoeff_nonlin[pfc.lincoeff != 0] = (expcoeff[pfc.lincoeff != 0] - 1)/pfc.lincoeff[pfc.lincoeff != 0]
expcoeff_nonlin.min(), expcoeff_nonlin.max()

In [ ]:
(np.exp(0.00000001*0.1) - (1+0.00000001*0.1))/0.00000001**2

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Phi")
plt.contourf(pfc.phi, pfc.nx, cmap='viridis')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Normalized Gaussian White Noise")
plt.contourf(pfc.GetEtaNoise(), pfc.nx, cmap='viridis')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
plt.contourf(pfc.k4, pfc.nx, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
plt.contourf(pfc.lincoeff, pfc.nx, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
pfc.TimeStep()

In [ ]:
pfc.phi_hat.shape

In [ ]:
pfc.phi_hat.shape

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Phi")
plt.contourf(pfc.phi, pfc.nx, cmap='viridis')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Normalized Gaussian White Noise")
plt.contourf(pfc.GetEtaNoise(), pfc.nx, cmap='viridis')
plt.colorbar()

plt.tight_layout()
plt.show()